In [1]:
from transformers import AutoProcessor, AutoModelForVision2Seq, Qwen2_5_VLForConditionalGeneration
from PIL import Image
import torch

# processor  = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
# model = AutoModelForVision2Seq.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# image= Image.open("Charts/253.png")
prompt = """
You are a reasoning model specialized in analyzing charts and their captions. Your task is to carefully examine the provided chart and caption, then clearly identify and list the attributes used in the chart.

When presented with a chart and its caption:
    1. Carefully inspect the chart provided.
    2. **Let's think step-by-step.** First, carefully inspect the provided chart and caption. Identify and list all attributes used in the chart. 
    3. **Briefly explain** how each attribute is represented or visualized in the chart (e.g., axis labels, legends, colors, data points).
    4. **Clearly separate** your response into two sections:
        a. "**Identified Attributes:**" (list attributes succinctly)
        b. "**Visualization Explanation:**" (briefly describe how each attribute is visualized)
    5. **Clearly summarize** the specific chart variables used (e.g., x-axis, y-axis, legend, color encoding) separately at the end of your response.

Important instructions for optimal performance:

1. Be concise and direct in your analysis.
2. Do not include unnecessary examples or additional context beyond what is provided.
3. Do not generate or assume any information not explicitly present in the provided chart and caption.
4. Leverage your internal chain-of-thought reasoning capability without explicit prompting for step-by-step reasoning.
5. Ensure your output is accurate, consistent, and directly based on the provided data only.
"""

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype=torch.float16, device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
image = Image.open("Charts/253.png").convert('RGB')
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": prompt}
        ]
    }
]
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Process the formatted text and the image together
inputs = processor(text=[text], images=[image], return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate the response
output = model.generate(**inputs, max_new_tokens=512)
result = processor.decode(output[0], skip_special_tokens=True)
print(result)

OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 7.65 GiB of which 183.06 MiB is free. Including non-PyTorch memory, this process has 7.42 GiB memory in use. Of the allocated memory 7.05 GiB is allocated by PyTorch, and 215.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)




Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


['The image is a line graph that illustrates the global emissions of carbon dioxide (CO₂) from various sectors over time, from 1970 to 2025. The y-axis represents the emissions in gigatons (Gt) of CO₂ per year, and the x-axis represents the years. The graph includes data for several categories:\n\n1. **Electricity plants**: This category shows a significant upward trend, indicating a substantial increase in CO₂ emissions from electricity generation.\n2. **Industry (excluding cement)**: This category also shows an upward trend but at a lower rate compared to electricity plants.\n3.']


In [6]:
prompt = """
You are a reasoning model specialized in analyzing charts and their captions. Your task is to carefully examine the provided chart and caption, then clearly identify and list the attributes used in the chart.

When presented with a chart and its caption:
    1. Carefully inspect the chart provided.
    2. **Let's think step-by-step.** First, carefully inspect the provided chart and caption. Identify and list all attributes used in the chart. 
    3. **Briefly explain** how each attribute is represented or visualized in the chart (e.g., axis labels, legends, colors, data points).
    4. **Clearly separate** your response into two sections:
        a. "**Identified Attributes:**" (list attributes succinctly)
        b. "**Visualization Explanation:**" (briefly describe how each attribute is visualized)
    5. **Clearly summarize** the specific chart variables used (e.g., x-axis, y-axis, legend, color encoding) separately at the end of your response.

Important instructions for optimal performance:

1. Be concise and direct in your analysis.
2. Do not include unnecessary examples or additional context beyond what is provided.
3. Do not generate or assume any information not explicitly present in the provided chart and caption.
4. Leverage your internal chain-of-thought reasoning capability without explicit prompting for step-by-step reasoning.
5. Ensure your output is accurate, consistent, and directly based on the provided data only.
"""
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "Charts/253.png",
            },
            {"type": "text", "text": prompt},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacity of 7.65 GiB of which 911.06 MiB is free. Including non-PyTorch memory, this process has 6.71 GiB memory in use. Of the allocated memory 5.41 GiB is allocated by PyTorch, and 1.13 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)